# Add Memory to OpenAI Functions Agent

This notebook goes over how to add memory to OpenAI Functions agent.

In [19]:
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper,
    SQLDatabase,
    SQLDatabaseChain,
)
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from langchain.utilities import BingSearchAPIWrapper
from langchain.tools import Tool
from dotenv import load_dotenv

load_dotenv()


True

In [20]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = BingSearchAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    #Tool(
    #    name="Calculator",
    #    func=llm_math_chain.run,
    #    description="useful for when you need to answer questions about math",
    #),
]

In [21]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [22]:
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    agent_kwargs=agent_kwargs, 
    memory=memory,
)

In [23]:
agent.run("hi")



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID dba8c17e047f0b408873b003708c134f in your message.).


Hello! How can I assist you today?

> Finished chain.


'Hello! How can I assist you today?'

In [26]:
agent.run("my name is bob")



> Entering new  chain...
Yes, I remember that your name is Bob. How can I assist you today, Bob?

> Finished chain.


'Yes, I remember that your name is Bob. How can I assist you today, Bob?'

In [27]:
agent.run("whats my name")



> Entering new  chain...
Your name is Bob.

> Finished chain.


'Your name is Bob.'

In [28]:
agent.run("what is today's date?")



> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': 'import datetime\n\ncurrent_date = datetime.date.today()\ncurrent_date'} because the `arguments` is not valid JSON.

In [29]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

agent_executor = create_python_agent(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors": True},
)


In [38]:
result = agent_executor.run("Using your python tool, can you please compute the 100th fibonacci number?")



> Entering new  chain...

Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(100)`




KeyboardInterrupt: 

In [36]:
result

'The current version of Python is 3.9.6.'